In [1]:
import tensorflow as tf
import numpy as np
import os
import random

tfk = tf.keras
tfkl = tfk.layers

In [2]:
# import splitfolders
# splitfolders.ratio("training_data_final", "split-data", seed=127, ratio=(0.9, 0.1, 0))

Copying files: 3542 files [00:20, 171.22 files/s]


In [2]:
# Random seed for reproducibility
seed = 127

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [4]:
dataset_base = "training_data_final"

## Splitting in training dataset and validation dataset

In [4]:
img_height = 96
img_width = 96

batch_size = 32
epochs = 200

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch = 64

data_gen = ImageDataGenerator( 
                              horizontal_flip = True,
                              rotation_range = 180,
                              fill_mode='reflect',
                              channel_shift_range=15,
                              brightness_range = [1.0,1.1],
                              zoom_range = [1.0,1.25],
                              preprocessing_function = tf.keras.applications.xception.preprocess_input)

validation_gen = ImageDataGenerator(preprocessing_function = tf.keras.applications.xception.preprocess_input)


train_gen = data_gen.flow_from_directory(directory="split-data/train",
                                                           target_size=(96,96),
                                                           color_mode='rgb',
                                                           classes=None,
                                                           class_mode='categorical',
                                                           batch_size=batch,
                                                           shuffle=True,
                                                           seed=seed) # set as training data

val_gen = validation_gen.flow_from_directory(directory="split-data/val",
                                                           target_size=(96,96),
                                                           color_mode='rgb',
                                                           classes=None,
                                                           class_mode='categorical',
                                                           batch_size=batch,
                                                           shuffle=True,
                                                           seed=seed) # set as validation data

Found 3183 images belonging to 8 classes.
Found 351 images belonging to 8 classes.


# Transfer learning

In [7]:
input_shape = (96,96,3)

In [8]:
# Import Xception base model
base_model = tfk.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=input_shape,
    include_top=False)  # Do not include the ImageNet classifier at the top.

#base_model.summary()
#tfk.utils.plot_model(base_model)

In [14]:
# freeze the base model
base_model.trainable = True

In [15]:
# Create a new model on top

inputs = tfk.Input(shape=input_shape)
x = base_model(inputs, training=True)
# x = tfkl.Resizing(img_height, img_width, interpolation='bicubic', name='Resize')

# # flattering
# x = tfkl.Flatten(name='Flattening')(x)

# # classifier with dropout, dense (256), dropout, dense (14)
# x = base_model(inputs)
x = tfkl.GlobalAveragePooling2D()(x)

x = tfkl.Dense(
    2048,                               
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

x = tfkl.Dropout(0.4, seed=seed)(x)

x = tfkl.Dense(
    2048,                                
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

x = tfkl.Dropout(0.4, seed=seed)(x)

outputs = tfkl.Dense(8, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
model = tfk.Model(inputs, outputs, name='base_model')

In [16]:

model.compile(optimizer=tfk.optimizers.Adam(),
              loss=tfk.losses.CategoricalCrossentropy(),
              metrics='accuracy')
model.summary()

Model: "base_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 xception (Functional)       (None, 3, 3, 2048)        20861480  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 2048)              4196352   
                                                                 
 dropout_4 (Dropout)         (None, 2048)              0         
                                                                 
 dense_7 (Dense)             (None, 2048)              4196352   
                                                        

In [14]:
# create class weights
class_distributions = []
for i in range(8):
  class_distributions.append(sum(1 for x in train_gen.labels if x == i))
  
weights = []
total = train_gen.labels.size

for i in range(8):
  weights.append((i,(1/class_distributions[i]) * (total/8.0)))

class_weights  = {cat:weight for (cat,weight) in weights}

In [15]:
class_weights

{0: 2.38248502994012,
 1: 0.8323744769874476,
 2: 0.8593412526997841,
 3: 0.8668300653594772,
 4: 0.8341194968553459,
 5: 1.9993718592964824,
 6: 0.8237577639751553,
 7: 0.8706236323851203}

In [17]:


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
batch_size= 64
epochs=300

# Train the model
tl_history = model.fit(
          x = train_gen,
          batch_size=batch_size,
          epochs = epochs,
          validation_data = val_gen,
          callbacks = callbacks).history


Epoch 1/300
50/50 [==============================] - 163s 3s/step - loss: 1.7207 - accuracy: 0.3918 - val_loss: 1.2013 - val_accuracy: 0.5157
Epoch 2/300
50/50 [==============================] - 213s 4s/step - loss: 1.1214 - accuracy: 0.6280 - val_loss: 0.8952 - val_accuracy: 0.6724
Epoch 3/300
50/50 [==============================] - 244s 5s/step - loss: 0.9250 - accuracy: 0.6871 - val_loss: 0.8962 - val_accuracy: 0.7151
Epoch 4/300
50/50 [==============================] - 238s 5s/step - loss: 0.7936 - accuracy: 0.7405 - val_loss: 0.6460 - val_accuracy: 0.7949
Epoch 5/300
50/50 [==============================] - 236s 5s/step - loss: 0.6891 - accuracy: 0.7848 - val_loss: 0.6729 - val_accuracy: 0.7778
Epoch 6/300
50/50 [==============================] - 242s 5s/step - loss: 0.6172 - accuracy: 0.7992 - val_loss: 0.6318 - val_accuracy: 0.7920
Epoch 7/300
50/50 [==============================] - 218s 4s/step - loss: 0.6412 - accuracy: 0.7999 - val_loss: 0.6024 - val_accuracy: 0.8120
Epoch 

In [27]:
# Evaluate on validation set
# Trainined with no data augmentation
#model_ = tfk.models.load_model("model/CNN_bn_dropout")
model_test_metrics = model.evaluate(val_gen, return_dict=True)

print()
print("Test metrics without data augmentation")
print(model_test_metrics)

6/6 [==============================] - 4s 649ms/step - loss: 0.4020 - accuracy: 0.9031

Test metrics without data augmentation
{'loss': 0.4019758105278015, 'accuracy': 0.9031339287757874}


In [28]:
model.save("model_Xception22/Xception_ftl22")

INFO:tensorflow:Assets written to: model_Xception22/Xception_ftl22\assets


INFO:tensorflow:Assets written to: model_Xception22/Xception_ftl22\assets
